[View in Colaboratory](https://colab.research.google.com/github/youqad/Neurorobotics_Intent-Recognition/blob/master/TP_Intent_Recognition.ipynb)

# TP: Intent recognition

# *Ex1*: Automatic detection of speaker’s intention from supra-segmental features

## 1. Extraction of prosodic features ($f_0$ and energy)

In [0]:
import urllib.request
import numpy as np
import random
import pandas as pd
from google.colab import files as google_files

In [0]:
def list_from_URL(file_URL, function_applied=None):
  lines_bytes = urllib.request.urlopen(file_URL).readlines()
  lines = []

  for line in lines_bytes:
    line = line.decode("utf-8").rstrip()
    
    if function_applied is not None:
      line = function_applied(line)
    
    lines.append(line)
   
  return lines

filenames = list_from_URL('https://raw.githubusercontent.com/youqad/Neurorobotics_Intent-Recognition/master/filenames.txt')
filenames = list(set(filenames))

In [0]:
# # /!\ NO NEED TO EXECUTE THIS CELL AGAIN !!!
# 
# files = []
# indices = []
# 
# for file in filenames:
# 
#     URL_f0 = 'https://raw.githubusercontent.com/youqad/Neurorobotics_Intent-Recognition/master/data_files/{}.f0'.format(file)
#     file_dicts = [{key:val for key, val in zip(['time', 'f0'], map(float, l.split()))} for l in list_from_URL(URL_f0)]
# 
#     URL_en = 'https://raw.githubusercontent.com/youqad/Neurorobotics_Intent-Recognition/master/data_files/{}.en'.format(file)
#     for l, d in zip(list_from_URL(URL_en), file_dicts):
#       d["file"] = file
#       d["en"] = float(l.split()[1])
# 
#     files.extend(file_dicts)
# 
# # How `files` looks like:
# # files = [ 
# #           {"file": "cy0001at", "time": 0.02, "f0": 0., "en": 0.},
# #           {"file": "cy0001at", "time": 1.28, "f0": 0., "en": 0.},
# #           ...
# #           {"file": "li1450at", "time": 0.02, "f0": 0., "en": 0.},
# #           {"file": "li1450at", "time": 1.56, "f0": 404., "en": 65.}
# #         ]
# 
# pd.DataFrame(files).to_csv('data.csv', encoding='utf-8', index=False) # To reuse it next time
# google_files.download('data.csv')

In [21]:
# loading training data
df = pd.read_csv('https://raw.githubusercontent.com/youqad/Neurorobotics_Intent-Recognition/master/data.csv').set_index('file')
df.head()

,en,f0,time
file,,,
cy0234pw,0.0,0.0,0.02
cy0234pw,0.0,0.0,0.04
cy0234pw,59.0,0.0,0.06
cy0234pw,59.0,0.0,0.08
cy0234pw,58.0,0.0,0.10


In [29]:
print(df.columns.values)

df.groupby('file').mean().head()

['en' 'f0' 'time']


,en,f0,time
file,,,
cy0001at,47.296875,110.609375,0.65
cy0002at,47.337079,105.640449,0.90
cy0005at,47.296875,110.609375,0.65
cy0006at,47.337079,105.640449,0.90
cy0007pw,52.313725,92.284314,1.03


In [30]:
df.groupby('file').max().head()

,en,f0,time
file,,,
cy0001at,73.0,402.0,1.28
cy0002at,76.0,430.0,1.78
cy0005at,73.0,402.0,1.28
cy0006at,76.0,430.0,1.78
cy0007pw,71.0,257.0,2.04


In [31]:
df.groupby('file').var().head()

,en,f0,time
file,,,
cy0001at,327.323165,27607.511657,0.138667
cy0002at,269.112360,27108.528345,0.267000
cy0005at,327.323165,27607.511657,0.138667
cy0006at,269.112360,27108.528345,0.267000
cy0007pw,228.455057,10372.542128,0.350200


In [32]:
df.groupby('file').median().head()

,en,f0,time
file,,,
cy0001at,41.5,0.0,0.65
cy0002at,41.0,0.0,0.90
cy0005at,41.5,0.0,0.65
cy0006at,41.0,0.0,0.90
cy0007pw,52.0,0.0,1.03


In [34]:
df.groupby('file').quantile([.25, .75]).head()

en     f0   time
file                             
cy0001at 0.25  40.0    0.0  0.335
         0.75  67.0  331.0  0.965
cy0002at 0.25  40.0    0.0  0.460
         0.75  62.0  251.0  1.340
cy0005at 0.25  40.0    0.0  0.335

In [38]:
agg_features = {
    'mean': 'mean',
    'max': 'max',
    'range': lambda x: max(x)-min(x),
    'var': 'var',
    'median': 'median',
    '1st_quartile': lambda x: x.quantile(.25),
    '3rd_quartile': lambda x: x.quantile(.75),
    'mean_absolute_local_derivate': lambda x: abs(x.diff()).mean()
}

df.groupby('file')['f0','en'].agg(agg_features)

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


mean               max        range                 var  \
                  f0         en     f0    en     f0    en            f0   
file                                                                      
cy0001at  110.609375  47.296875  402.0  73.0  402.0  73.0  27607.511657   
cy0002at  105.640449  47.337079  430.0  76.0  430.0  76.0  27108.528345   
cy0005at  110.609375  47.296875  402.0  73.0  402.0  73.0  27607.511657   
cy0006at  105.640449  47.337079  430.0  76.0  430.0  76.0  27108.528345   
cy0007pw   92.284314  52.313725  257.0  71.0  257.0  71.0  10372.542128   
cy0008pw   78.431373  47.725490  250.0  70.0  250.0  70.0   9930.090196   
cy0009pw   69.065789  49.473684  243.0  74.0  243.0  74.0   8927.182281   
cy0010pw   29.196078  46.049020  221.0  77.0  221.0  77.0   4696.178994   
cy0011pw  110.743590  53.653846  230.0  71.0  230.0  71.0   9290.400932   
cy0012pw   74.539474  50.250000  224.0  67.0  224.0  67.0   7363.451754   
cy0013pw   81.343137  48.960784  597.0  69.0  597.0  69.0  11985.257329   
cy0014ap  114.578431  53.558824  321.0  74.0  321.0  74.0  13517.612599   
cy0015ap  107.976562  52.109375  375.0  73.0  375.0  73.0  12282.180549   
cy0016ap   87.125714  48.354286  399.0  76.0  399.0  76.0  20819.478358   
cy0017ap  143.225490  51.931373  528.0  73.0  528.0  73.0  27929.225878   
cy0018ap  101.934211  48.078947  505.0  75.0  505.0  75.0  27713.342281   
cy0019ap  123.401961  53.411765  448.0  72.0  448.0  72.0  15774.599204   
cy0020ap  128.509804  53.637255  435.0  77.0  435.0  77.0  20712.628616   
cy0021ap  129.179775  51.887640  477.0  78.0  477.0  78.0  30549.717314   
cy0022ap  128.205882  50.411765  488.0  78.0  488.0  78.0  28859.373034   
cy0023ap   79.775281  45.820225  510.0  77.0  510.0  77.0  23343.562564   
cy0024ap  130.674157  50.584270  448.0  72.0  448.0  72.0  26981.835802   
cy0025ap  138.426966  51.719101  471.0  76.0  471.0  76.0  24146.383810   
cy0026at  144.196078  47.862745  453.0  73.0  453.0  73.0  39122.680784   
cy0027at  189.872549  52.823529  487.0  75.0  487.0  75.0  41119.023199   
cy0028at  170.987805  52.024390  501.0  80.0  501.0  80.0  42613.715899   
cy0029at  111.123596  48.112360  496.0  76.0  496.0  76.0  31597.655005   
cy0030at  190.562500  52.000000  474.0  75.0  474.0  75.0  45064.535714   
cy0031at  149.015625  48.031250  481.0  75.0  481.0  75.0  43398.777530   
cy0111ap  109.398438  51.445312  427.0  77.0  427.0  77.0  17347.895116   
...              ...        ...    ...   ...    ...   ...           ...   
li1421at  120.833333  51.509804  514.0  81.0  514.0  81.0  30060.615512   
li1422at  151.887640  51.460674  508.0  79.0  508.0  79.0  39705.100868   
li1423at   97.071429  52.778571  514.0  81.0  514.0  81.0  29858.743063   
li1424at  181.313725  56.852941  509.0  78.0  509.0  78.0  35839.148127   
li1425at  109.669725  51.779817  544.0  83.0  544.0  83.0  30612.612130   
li1426at  111.078431  50.970588  553.0  80.0  553.0  80.0  33590.271015   
li1427at  135.191011  50.696629  443.0  77.0  443.0  77.0  33501.633555   
li1428at   99.523438  49.937500  584.0  78.0  584.0  78.0  30401.904958   
li1429at  104.977528  50.370787  495.0  80.0  495.0  80.0  28627.340398   
li1430at  161.556522  56.373913  500.0  80.0  500.0  80.0  33990.477040   
li1431at   84.960526  46.802632  581.0  76.0  581.0  76.0  27200.545088   
li1432at  117.344086  48.849462  551.0  83.0  551.0  83.0  38972.684666   
li1433at  138.547826  52.321739  514.0  80.0  514.0  80.0  34515.828833   
li1434at  142.234375  51.820312  472.0  77.0  472.0  77.0  32314.637549   
li1435at  122.008696  54.756522  493.0  83.0  493.0  83.0  33095.710450   
li1436at  135.970588  51.823529  464.0  82.0  464.0  82.0  30828.286255   
li1437at   96.225490  50.029412  455.0  81.0  455.0  81.0  25101.562512   
li1438at  151.914062  53.828125  461.0  79.0  461.0  79.0  36647.779958   
li1439at  129.647059  51.705882  537.0  85.0  537.0  85.0  38897.676179   
li1440at   92.304348  48.765217  486.0  8

In [0]:
def statistics(data_matrix):
  output = []
  
  for file in data_matrix:
    mean = np.mean(file[:,1])
    
    maximum = np.max(file[:,1]) 
    range = maximum - np.min(file[:,1])
    
    variance = np.var(file[:,1])
    median = np.median(file[:,1])
    
    first_quartile = np.percentile(file[:,1],25)
    third_quartile = np.percentile(file[:,1],75)
    mean_absolute_local_derivate = np.mean(np.abs(np.diff(file[:,1])))
    
    output.append([mean,maximum,range,variance,median,first_quartile,third_quartile,mean_absolute_local_derivate])
    
  return np.array(output)

features = {}
features['f0'] = []
features['en'] = []


for ext in ['f0', 'en']:
  features[ext] = statistics(files[ext])

In [5]:
features

{'en': array([[60.94252874, 83.        , 83.        , ..., 59.        ,
         63.5       ,  3.06976744],
        [49.5826087 , 86.        , 86.        , ..., 40.        ,
         64.5       ,  3.66666667],
        [70.02      , 91.        , 91.        , ..., 57.        ,
         83.        ,  4.64646465],
        ...,
        [66.72580645, 85.        , 85.        , ..., 55.        ,
         82.        ,  4.45901639],
        [50.609375  , 83.        , 83.        , ..., 40.        ,
         66.5       ,  3.7480315 ],
        [55.31304348, 81.        , 81.        , ..., 42.        ,
         69.5       ,  3.92982456]]),
 'f0': array([[ 34.94252874, 258.        , 258.        , ...,   0.        ,
           0.        ,  12.51162791],
        [114.76521739, 508.        , 508.        , ...,   0.        ,
         337.        ,  29.9122807 ],
        [115.3       , 501.        , 501.        , ...,   0.        ,
         192.        ,  38.32323232],
        ...,
        [105.01612903, 2

In [0]:
# Question 3

## Explanation: A sound can be either voiced, like a vowel, or unvoiced, like a consonant. We want to see which of them contains more intention
 
## ps: change "data" by the correct name

voiced = data.loc[data[:,1] != 0]
unvoiced = data.loc[data[:,1] == 0]
  

# Question 4


tot_sz = length(data)

train_sz = round(tot_sz*0.6)
random.shuffle(data)
train_data = data[:train_sz]
test_data = data[train_sz:]
  

  


In [0]:
x = np.array([[1, 3], 
              [0, 5],
              [0, 5]])


#np.mean(np.abs(np.diff(x, axis=0)), axis=0)
np.mean(np.abs(np.diff(x, axis=0)), axi

array([0.5, 1. ])